- `actor`: ID of chimpanzee (1 - 7)
- `condition`: `1` if partner is sitting on opposite side else `0` for lone experiment
- `prosoc_left`: `1` if two food items on left else `0` for two food items on right side
- `pulled_left`: `1` if left lever pulled else `0` for right lever pulled

### Problem
- If humans participate GIVEN another human is on the opposite side of the table, the lever linked to the two food items is always picked (the prosocial option)
- Question: *Do chimpanzees behave the same? If not, describe their behavior (interpret model results).*
- It seems we wish to predict if a chimpanzee will pick the more prosocial option **if** there is a partner chimp sitting opposite.
    - So we want `prosoc_left == pulled_left & condition == 1` > `prosoc_left == pulled_left & condition == 0`

### Requirements
- Data visualization
- Choice of model
- Model fitting and selection (AIC)
- Diagnostics (overdisperson checking)

In [2]:
str(df)

'data.frame':	504 obs. of  4 variables:
 $ actor      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ condition  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ prosoc_left: int  0 0 1 0 1 1 1 1 0 0 ...
 $ pulled_left: int  0 1 0 0 1 1 0 0 0 0 ...


In [3]:
head(df)

actor,condition,prosoc_left,pulled_left
1,0,0,0
1,0,0,1
1,0,1,0
1,0,0,0
1,0,1,1
1,0,1,1


Compare `is_prosoc` wrt `condition`?

In [5]:
head(df)

actor,condition,prosoc_left,pulled_left,is_prosoc
1,0,0,0,1
1,0,0,1,0
1,0,1,0,0
1,0,0,0,1
1,0,1,1,1
1,0,1,1,1


Frequentist Estimation from Observed Results

In [6]:
single = df[which(df$condition == 0),]
pair = df[which(df$condition == 1),]

In [7]:
sum(prop.table(table(single$actor, single$is_prosoc), 1)[,2] < prop.table(table(pair$actor, pair$is_prosoc), 1)[,2])
sum(prop.table(table(single$actor, single$is_prosoc), 1)[,2] == prop.table(table(pair$actor, pair$is_prosoc), 1)[,2])
sum(prop.table(table(single$actor, single$is_prosoc), 1)[,2] > prop.table(table(pair$actor, pair$is_prosoc), 1)[,2])

[1] 3

[1] 3

[1] 1

- 4 actors chose the more prosocial option
- 3 actors chose the non prosocial option
- Roughly speaking, chimps do indeed choose the more prosocial option when there is a partner chimp sitting on the opposite side.
    - Now, we need to test this relationship using regression

In [8]:
t = table(df$pulled_left, df$actor)
summary(t)

Number of cases in table: 504 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 129.11, df = 6, p-value = 1.979e-25

### Contingency Table
- Two Factors, we wish to test independence

Control group:

In [9]:
cta = xtabs(pulled_left ~ actor + prosoc_left, df, subset=(condition==0))
cta
summary(cta)

     prosoc_left
actor  0  1
    1  6  9
    2 18 18
    3  5 11
    4  6  9
    5  6 10
    6 14 11
    7 14 15

Call: xtabs(formula = pulled_left ~ actor + prosoc_left, data = df, 
    subset = (condition == 0))
Number of cases in table: 152 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 3.585, df = 6, p-value = 0.7326

We observe that the chimps will pull the left lever independently of actor and regardless of prosocial option.

Partner opposite:

In [10]:
ctb = xtabs(pulled_left ~ actor + prosoc_left, df, subset=(condition==1))
summary(ctb)

Call: xtabs(formula = pulled_left ~ actor + prosoc_left, data = df, 
    subset = (condition == 1))
Number of cases in table: 140 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 4.725, df = 6, p-value = 0.5795
	Chi-squared approximation may be incorrect

Here, it seems that the left lever being pulled is independent.

#### Test for independence for prosocial chimps: Side of Prosocial Option + Actor

In [11]:
ctb = xtabs(is_prosoc ~ pulled_left + actor, df, subset=(condition==1))
ctb
summary(ctb)

           actor
pulled_left  1  2  3  4  5  6  7
          0 13  0 15 16 13  8  1
          1 10 18  6  8  9 11 18

Call: xtabs(formula = is_prosoc ~ pulled_left + actor, data = df, subset = (condition == 
    1))
Number of cases in table: 146 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 40.36, df = 6, p-value = 3.879e-07

In [12]:
ctb = xtabs(is_prosoc ~ prosoc_left + actor, df)
ctb
summary(ctb)

           actor
prosoc_left  1  2  3  4  5  6  7
          0 25  0 28 28 25 12  5
          1 19 36 17 17 19 22 33

Call: xtabs(formula = is_prosoc ~ prosoc_left + actor, data = df)
Number of cases in table: 286 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 62.21, df = 6, p-value = 1.599e-11

#### Finding a model for experiments WITH A PARTNER ACROSS

In [13]:
m = glm(is_prosoc ~ actor + pulled_left + prosoc_left, pair, family=poisson)
summary(m)


Call:
glm(formula = is_prosoc ~ actor + pulled_left + prosoc_left, 
    family = poisson, data = pair)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1975  -0.9892   0.3531   0.5171   0.6935  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept) -0.56583    0.23835  -2.374   0.0176 *
actor2      -0.27321    0.33464  -0.816   0.4143  
actor3      -0.08236    0.30390  -0.271   0.7864  
actor4       0.04939    0.29315   0.168   0.8662  
actor5      -0.04294    0.29828  -0.144   0.8855  
actor6      -0.19884    0.31165  -0.638   0.5235  
actor7      -0.21791    0.32858  -0.663   0.5072  
pulled_left  0.04975    0.20361   0.244   0.8070  
prosoc_left  0.18384    0.16996   1.082   0.2794  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 159.38  on 251  degrees of freedom
Residual deviance: 156.50  on 243  degrees of freedom
AIC: 466.5

Number of Fi

In [14]:
mFinal = step(m, trace=FALSE)
summary(mFinal)


Call:
glm(formula = is_prosoc ~ 1, family = poisson, data = pair)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0764  -1.0764   0.5004   0.5004   0.5004  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.54582    0.08276  -6.595 4.25e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 159.38  on 251  degrees of freedom
Residual deviance: 159.38  on 251  degrees of freedom
AIC: 453.38

Number of Fisher Scoring iterations: 5


In [61]:
coefs = summary(mFinal)$coefficients
beta.0 = coefs[1]
s0 = coefs[1,2]
alpha = qnorm(0.975)

CI.beta0 = beta.0 + c(-1, 1) * alpha * s0
CI.beta0

[1] -0.7080302 -0.3836148

In [95]:
N = dim(pair)[1]
p = 2

# phi.hat is the dispersion parameter 
phi.hat = sum(residuals(mFinal, "pearson")^2) / (N - p)
cbind(phi.hat)

phi.hat
0.424


In [60]:
summary(table(pair$is_prosoc, pair$actor))

Number of cases in table: 252 
Number of factors: 2 
Test for independence of all factors:
	Chisq = 3.517, df = 6, p-value = 0.7417

#### Including all the data...

In [81]:
test = glm(is_prosoc ~ actor + pulled_left + prosoc_left + condition, df, family=poisson)
summary(test)


Call:
glm(formula = is_prosoc ~ actor + pulled_left + prosoc_left + 
    condition, family = poisson, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2025  -1.0192   0.3327   0.5486   0.8062  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -6.699e-01  1.836e-01  -3.648 0.000264 ***
actor2      -2.091e-01  2.379e-01  -0.879 0.379547    
actor3       2.351e-02  2.122e-01   0.111 0.911795    
actor4       2.350e-02  2.122e-01   0.111 0.911840    
actor5       1.002e-05  2.132e-01   0.000 0.999962    
actor6      -2.608e-01  2.300e-01  -1.134 0.256862    
actor7      -1.534e-01  2.304e-01  -0.666 0.505478    
pulled_left  1.483e-02  1.384e-01   0.107 0.914668    
prosoc_left  2.794e-01  1.212e-01   2.305 0.021163 *  
condition    4.269e-02  1.185e-01   0.360 0.718592    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 324.09  

In [82]:
testStep = step(test, trace=FALSE)
summary(testStep)


Call:
glm(formula = is_prosoc ~ prosoc_left, family = poisson, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1374  -0.9880   0.4062   0.4062   0.6408  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.71724    0.09017  -7.955  1.8e-15 ***
prosoc_left  0.28157    0.11944   2.357   0.0184 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 324.09  on 503  degrees of freedom
Residual deviance: 318.47  on 502  degrees of freedom
AIC: 894.47

Number of Fisher Scoring iterations: 5


In [84]:
N2 = length(df$actor)
p2 = length(test$coef)

# phi.hat is the dispersion parameter 
phi.hat2 = sum(residuals(test, "pearson")^2) / (N2 - p2)
phi.hat2

[1] 0.4357657

#### Testing for independence between actors pulling the left lever when they are by themselves (control). 

In [25]:
m2 = glm(pulled_left ~ actor, single, family=poisson)
summary(m2)


Call:
glm(formula = pulled_left ~ actor, family = poisson, data = single)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2693  -0.9129   0.0000   0.3438   0.7644  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -8.755e-01  2.582e-01  -3.391 0.000697 ***
actor2       8.755e-01  3.073e-01   2.849 0.004389 ** 
actor3       6.454e-02  3.594e-01   0.180 0.857486    
actor4       4.023e-16  3.651e-01   0.000 1.000000    
actor5       6.454e-02  3.594e-01   0.180 0.857486    
actor6       5.108e-01  3.266e-01   1.564 0.117797    
actor7       6.592e-01  3.180e-01   2.073 0.038186 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 153.69  on 251  degrees of freedom
Residual deviance: 135.20  on 245  degrees of freedom
AIC: 453.2

Number of Fisher Scoring iterations: 5


In [26]:
m2Final = step(m2, trace=FALSE)
summary(m2Final)


Call:
glm(formula = pulled_left ~ actor, family = poisson, data = single)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2693  -0.9129   0.0000   0.3438   0.7644  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -8.755e-01  2.582e-01  -3.391 0.000697 ***
actor2       8.755e-01  3.073e-01   2.849 0.004389 ** 
actor3       6.454e-02  3.594e-01   0.180 0.857486    
actor4       4.023e-16  3.651e-01   0.000 1.000000    
actor5       6.454e-02  3.594e-01   0.180 0.857486    
actor6       5.108e-01  3.266e-01   1.564 0.117797    
actor7       6.592e-01  3.180e-01   2.073 0.038186 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 153.69  on 251  degrees of freedom
Residual deviance: 135.20  on 245  degrees of freedom
AIC: 453.2

Number of Fisher Scoring iterations: 5


### So can we determine if `pulled_left` given attributes?

In [19]:
t = glm(pulled_left ~ actor + condition + prosoc_left, df, family=poisson)
summary(t)


Call:
glm(formula = pulled_left ~ actor + condition + prosoc_left, 
    family = poisson, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.43748  -0.86223  -0.03281   0.48835   1.01758  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -9.590e-01  2.020e-01  -4.747 2.07e-06 ***
actor2       8.755e-01  2.173e-01   4.029 5.61e-05 ***
actor3      -1.823e-01  2.708e-01  -0.673 0.500772    
actor4      -1.823e-01  2.708e-01  -0.673 0.500772    
actor5      -5.232e-12  2.582e-01   0.000 1.000000    
actor6       4.274e-01  2.347e-01   1.821 0.068541 .  
actor7       7.577e-01  2.213e-01   3.424 0.000616 ***
condition   -8.224e-02  1.171e-01  -0.702 0.482647    
prosoc_left  2.339e-01  1.178e-01   1.985 0.047125 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 318.76  on 503  degrees of freedom
Residual deviance: 262.67  on 

In [20]:
tstep = step(t, trace=FALSE)
summary(tstep)


Call:
glm(formula = pulled_left ~ actor + prosoc_left, family = poisson, 
    data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.40882  -0.85808   0.00763   0.52214   0.98831  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -9.993e-01  1.941e-01  -5.149 2.62e-07 ***
actor2       8.755e-01  2.173e-01   4.029 5.61e-05 ***
actor3      -1.823e-01  2.708e-01  -0.673 0.500772    
actor4      -1.823e-01  2.708e-01  -0.673 0.500772    
actor5       2.457e-16  2.582e-01   0.000 1.000000    
actor6       4.274e-01  2.347e-01   1.821 0.068541 .  
actor7       7.577e-01  2.213e-01   3.424 0.000616 ***
prosoc_left  2.339e-01  1.178e-01   1.985 0.047125 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 318.76  on 503  degrees of freedom
Residual deviance: 263.16  on 496  degrees of freedom
AIC: 863.16

Number of Fisher Scoring itera

Now, it seems we can determine if a chimp pulled left given the ID of chimp AND the side of the prosocial option.

# Realistic Predictors???

In [21]:
t2 = glm(is_prosoc ~ actor + condition + prosoc_left, df, family=poisson)
summary(t2)
t2step = step(t2, trace=FALSE)
summary(t2step)


Call:
glm(formula = is_prosoc ~ actor + condition + prosoc_left, family = poisson, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2061  -1.0145   0.3399   0.5455   0.7998  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -6.643e-01  1.761e-01  -3.773 0.000161 ***
actor2      -2.007e-01  2.247e-01  -0.893 0.371895    
actor3       2.247e-02  2.120e-01   0.106 0.915584    
actor4       2.247e-02  2.120e-01   0.106 0.915584    
actor5       1.078e-14  2.132e-01   0.000 1.000000    
actor6      -2.578e-01  2.283e-01  -1.129 0.258835    
actor7      -1.466e-01  2.215e-01  -0.662 0.507973    
condition    4.196e-02  1.183e-01   0.355 0.722768    
prosoc_left  2.816e-01  1.194e-01   2.357 0.018401 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 324.09  on 503  degrees of freedom
Residual deviance: 315.04  on 495  degrees


Call:
glm(formula = is_prosoc ~ prosoc_left, family = poisson, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1374  -0.9880   0.4062   0.4062   0.6408  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.71724    0.09017  -7.955  1.8e-15 ***
prosoc_left  0.28157    0.11944   2.357   0.0184 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 324.09  on 503  degrees of freedom
Residual deviance: 318.47  on 502  degrees of freedom
AIC: 894.47

Number of Fisher Scoring iterations: 5


In [22]:
t3 = glm(is_prosoc ~ actor * condition * prosoc_left, df, family=poisson)
summary(t3)
summary(step(t3,trace=FALSE))


Call:
glm(formula = is_prosoc ~ actor * condition * prosoc_left, family = poisson, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3333  -0.8165   0.0000   0.3798   1.9728  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)  
(Intercept)                  -4.055e-01  2.887e-01  -1.405   0.1601  
actor2                       -1.790e+01  1.348e+03  -0.013   0.9894  
actor3                        8.004e-02  4.003e-01   0.200   0.8415  
actor4                        5.268e-16  4.082e-01   0.000   1.0000  
actor5                        8.256e-16  4.082e-01   0.000   1.0000  
actor6                       -1.099e+00  5.774e-01  -1.903   0.0571 .
actor7                       -1.099e+00  5.774e-01  -1.903   0.0571 .
condition                     8.004e-02  4.003e-01   0.200   0.8415  
prosoc_left                  -2.877e-01  4.410e-01  -0.652   0.5141  
actor2:condition             -8.004e-02  1.906e+03   0.000   1.0000  
a


Call:
glm(formula = is_prosoc ~ actor + prosoc_left + actor:prosoc_left, 
    family = poisson, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3540  -0.9718   0.0000   0.3438   1.4920  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -3.646e-01  2.000e-01  -1.823 0.068271 .  
actor2             -1.794e+01  9.529e+02  -0.019 0.984981    
actor3              1.133e-01  2.752e-01   0.412 0.680441    
actor4              1.133e-01  2.752e-01   0.412 0.680441    
actor5              7.863e-16  2.828e-01   0.000 1.000000    
actor6             -7.340e-01  3.512e-01  -2.090 0.036622 *  
actor7             -1.609e+00  4.899e-01  -3.285 0.001019 ** 
prosoc_left        -2.744e-01  3.044e-01  -0.902 0.367215    
actor2:prosoc_left  1.858e+01  9.529e+02   0.019 0.984446    
actor3:prosoc_left -2.246e-01  4.326e-01  -0.519 0.603730    
actor4:prosoc_left -2.246e-01  4.326e-01  -0.519 0.603730    
actor5:prosoc_left 